In [1]:
import pandas as pd
import os

def load_and_clean(path, product_name):
    df = pd.read_csv(path)

    # Rename Flipkart exported columns
    df = df.rename(columns={
        "XQDdHH": "rating",
        "z9E0IG": "review_title",
        "ZmyHeo": "review_text",
        "_2NsDsF": "reviewer_name",
        "MztJPv": "verified_purchase",
        "_2NsDsF 2": "review_date",
        "tl9VpF": "helpful_upvotes",
        "tl9VpF 2": "helpful_total"
    })

    # Keep only rows with review text
    df = df.dropna(subset=["review_text"])

    # Convert rating to numeric
    df["rating"] = pd.to_numeric(df["rating"], errors="coerce")

    # Verified buyer flag
    df["verified_purchase"] = df["verified_purchase"].astype(str)
    df["verified_purchase"] = df["verified_purchase"].apply(
        lambda x: True if "Certified Buyer" in x else False
    )

    # Keep raw date text
    df["review_date_raw"] = df["review_date"]

    # Try to parse dates (but do NOT leave NaT)
    d1 = pd.to_datetime(df["review_date_raw"], errors="coerce", dayfirst=True)
    d2 = pd.to_datetime(df["review_date_raw"], format="%d %b, %Y", errors="coerce")

    # Use whichever parse worked
    df["review_date"] = d1.fillna(d2)

    # Fill any remaining missing dates with median date
    if df["review_date"].isna().sum() > 0:
        median_date = df["review_date"].dropna().median()
        df["review_date"] = df["review_date"].fillna(median_date)

    # Add product name
    df["product"] = product_name

    return df


# -----------------------------
# File Paths
# -----------------------------
path_s24_pos = r"C:/Users/Aakash/Desktop/ECOM-- SENTIMENT/Data/Raw/flipkart _samsung_s24.csv"
path_ip15_pos = r"C:/Users/Aakash/Desktop/ECOM-- SENTIMENT/Data/Raw/flipkart_iphone_15.csv"
path_s24_neg = r"C:/Users/Aakash/Desktop/ECOM-- SENTIMENT/Data/Raw/flipkart_SamsungS24_negative.csv"
path_ip15_neg = r"C:/Users/Aakash/Desktop/ECOM-- SENTIMENT/Data/Raw/flipkart_Iphone15_negative.csv"

# -----------------------------
# Load All Datasets
# -----------------------------
df_s24_pos = load_and_clean(path_s24_pos, "Samsung S24")
df_ip15_pos = load_and_clean(path_ip15_pos, "iPhone 15")
df_s24_neg = load_and_clean(path_s24_neg, "Samsung S24")
df_ip15_neg = load_and_clean(path_ip15_neg, "iPhone 15")

# Combine
df_all = pd.concat([df_s24_pos, df_ip15_pos, df_s24_neg, df_ip15_neg], ignore_index=True)

# -----------------------------
# Save
# -----------------------------
os.makedirs("Data/Processed", exist_ok=True)
df_all.to_csv("Data/Processed/flipkart_cleaned.csv", index=False)

print("Saved cleaned dataset with", len(df_all), "rows.")


C:\Users\Aakash\AppData\Local\Temp\ipykernel_10524\3019553400.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d1 = pd.to_datetime(df["review_date_raw"], errors="coerce", dayfirst=True)
C:\Users\Aakash\AppData\Local\Temp\ipykernel_10524\3019553400.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d1 = pd.to_datetime(df["review_date_raw"], errors="coerce", dayfirst=True)
C:\Users\Aakash\AppData\Local\Temp\ipykernel_10524\3019553400.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d1 = pd.to_datetime(df["review_date_raw"], errors="coerce", dayfirst=True)
C:\Users\A

Saved cleaned dataset with 1623 rows.
